In [1]:
from datasets import load_dataset
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, MT5Config
import torch
from transformers.utils import is_torch_fx_proxy

from noise_functions.MT6NoiseFunction import MT6NoiseFunction

In [2]:
def model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024 ** 2
    return size_all_mb

In [4]:
from transformers import MT5TokenizerFast

tok_en = MT5TokenizerFast.from_pretrained("nikodallanoce/mt5-cc4-vanilla-32k-5")

cuda_dev = "cpu"

model = MT5ForConditionalGeneration.from_pretrained(
    "/home/n.dallanoce/PyCharm/pretraining/weights/mt6_loss/checkpoint-5120")
model = model.to(cuda_dev)
model.train(False)
print(model_size(model))

OSError: Can't load the configuration of '/home/n.dallanoce/PyCharm/pretraining/weights/mt6_loss/checkpoint-5120'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/home/n.dallanoce/PyCharm/pretraining/weights/mt6_loss/checkpoint-5120' is the correct path to a directory containing a config.json file

In [9]:
dataset = load_dataset("cc100", lang="en",
                                cache_dir="/data/n.dallanoce/cc100/huggingface",
                                split=f"train[{4096}:{4096*2}]",
                                verification_mode='no_checks')


KeyboardInterrupt



In [95]:
index = 111
sent = dataset[index]['text']
src_sent, tgt_sent = MT6NoiseFunction().compute_for_mt5(sent, seed=index, noise_density=0.35)
print(f"original: {sent} \n \nsource: {src_sent}")

original: Read about what EEI’s International Programs has accomplished in the first-half of 2018
 
 
source: <extra_id_0> what EEI’s International <extra_id_1> has accomplished <extra_id_2> of <extra_id_3>


In [8]:
index = 111
sent = dataset[index]['text']
src_sent, tgt_sent = MT6NoiseFunction().compute(sent, seed=index)
print(f"original: {sent} \n \nsource: {src_sent}")

NameError: name 'dataset' is not defined

In [4]:
input_ids = tok_en(src_sent, return_tensors="pt").input_ids
sequence_ids = model.generate(input_ids.to(cuda_dev), max_length=64)
sequences = tok_en.batch_decode(sequence_ids, skip_special_tokens=False)
print(f"prediction: {sequences} \n \nlabel: {tgt_sent}")

NameError: name 'src_sent' is not defined